In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print("% of women who survived:", rate_men)

In [ ]:
train_data.shape

In [ ]:
train_data.info()

In [ ]:
def print_all_info(data):
    for i in data.select_dtypes(exclude='object').columns:
        print('_________________________________________________________')
        print(i)
        print(data.groupby(['Survived'])[i].describe())
        
print_all_info(train_data)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

sns.set_style('whitegrid')
plt.rc('patch', edgecolor='black')
sns.set_context("notebook", font_scale=1.3, rc={"lines.linewidth": 1.5})

sns.factorplot('Embarked','Survived',data=train_data)

In [ ]:
train_data = pd.get_dummies(train_data, columns=["Embarked"])

test_data = pd.get_dummies(test_data, columns=["Embarked"])
train_data.info()
print_all_info(train_data)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# y = train_data["Survived"]
# features = ["Pclass", "Sex", "SibSp", "Parch"]
# X = pd.get_dummies(train_data[features])
# X_test = pd.get_dummies(test_data[features])

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(X,y)
# predictions = model.predict(X_test)

# output = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": predictions})
# output.to_csv("submission.csv", index=False)
# print("Submission saved!")



In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

write_submission = True

y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked_C", "Embarked_S", "Embarked_Q"]
X = pd.get_dummies(train_data[features])
# X_test = pd.get_dummies(test_data[features])
# X = pd.get_dummies(train_data[features])
# y = train_data["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)




reg_params = [0.001, 0.01, 0.1, 1.0, 2.0, 4.0,8.0,16.0,32.0]

for reg in reg_params:
    model = svm.SVC(probability=True, kernel='rbf', C=reg)
    model.fit(X_train, y_train)
    train_predictions = model.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("score: ", acc, " reg: ", reg)


model = svm.SVC(probability=True, kernel='rbf', C=16)
model.fit(X_train, y_train)
test_predictions = model.predict(pd.get_dummies(test_data[features]))

if write_submission:
    output = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": test_predictions})
    output.to_csv("submission.csv", index=False)
    print("Submission saved!")